# Aave Subgraphs
https://github.com/aave/protocol-subgraphs

# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from pathlib import Path

path = Path('/content/drive/MyDrive/colab_notebooks/algovera/defi')

In [3]:
import json

with open(path/"DataGathering/creds.json") as f:


    creds = json.loads(f.read())["ethkey"]

# API Calls

In [4]:
import json
from datetime import datetime, timedelta, timezone
from tqdm import tqdm
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


# Turn timestamp into block number
def get_block(s: requests.Session, timestamp: int) -> str:
    ETHERSCAN_KEY = ''
    ETHERSCAN_KEY_API_URL = f'https://api.etherscan.io/api?module=block&action=getblocknobytime&timestamp={timestamp}&closest=before&apikey={creds}'

    block_response = s.request('GET', ETHERSCAN_KEY_API_URL)
    block = block_response.json()['result']

    return block

#https://api.thegraph.com/subgraphs/name/aave/protocol-v2
#https://api.thegraph.com/subgraphs/name/aave/aave-v2-matic



def aave_scraper(dt):

    aaveV2_api_url = 'https://api.thegraph.com/subgraphs/name/aave/protocol-v2'

    retries = Retry(total=5, backoff_factor=2, status_forcelist=[429, 502, 503, 504])
    s = requests.Session()
    s.mount('https://', HTTPAdapter(max_retries=retries))
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}


    block = get_block(s, int(dt))
    body_AAVE = {
        "query": "{ reserves (first: 100,block: {number: %s})  { id symbol aEmissionPerSecond decimals totalATokenSupply price {priceInEth}"
                    " vEmissionPerSecond pool {id} totalCurrentVariableDebt utilizationRate totalLiquidity liquidityRate variableBorrowRate availableLiquidity}}" % block}
    aave_json_file = json.dumps(body_AAVE)
    response = s.request('POST', aaveV2_api_url, headers=headers, data=aave_json_file)

    return response


# Time Range

In [21]:
t = int(datetime.timestamp(datetime.now()));t

1655885309

In [7]:
import pandas as pd

date = datetime.fromtimestamp(1652207400, tz=timezone.utc)
end_date = datetime.fromtimestamp(t, tz=timezone.utc)

dates = pd.date_range(date,end_date,freq="30T")

timestamps = [pd.Timestamp(d).timestamp() for d in dates]

In [8]:
len(timestamps)

2041

# Main Loop

In [14]:
coins_id = {'TUSD': 'true-usd', 'DAI': 'dai', 'USDC': 'usd-coin', 'USDT': 'tether', 'WETH': 'weth'}

def fetch_data(timestamps):
    for d in tqdm(timestamps):

        data = list()
        kickout = list()

        response = aave_scraper(d)

        try:

            for token in response.json()['data']['reserves']:

                    data.append(token)


            df = pd.DataFrame(data)
            
            df["timestamp"] = int(d)

            df.to_csv(path/f"DataGathering/aaveV2/aave_mainnet_May-June22/{int(d)}_aave.csv",index=False)

        except KeyError:
            kickout.append(d)

    return kickout


In [16]:
kickout = [True]
i = 0
while kickout:

    if not kickout:
        kickout = fetch_data(kickout)
    else:
        kickout = fetch_data(timestamps)

    print(len(kickout))
    i+=1

100%|██████████| 2041/2041 [20:52<00:00,  1.63it/s]

0


In [18]:
t

1655880441.461917

In [23]:
1655879400-1655885309

-5909